## Transfer learning by fine tuning the VGG19 model 

NOTE THE FOLLOWING IN FINE TUNING:
<br>
> - We remove the last softmax layer corresponding to imagenet dataset (include_top=False)
> - We use small learning rates to train 
> - We freeze a large number of layers

In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D

/Users/prachigarg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam
from keras.applications.vgg19 import VGG19

In [3]:
import numpy as np
from keras.preprocessing.image import img_to_array, load_img
import os

In [4]:
def load_images_from_folder(direct):
    images = []
    y = []
    for filename in os.listdir(direct):
        image = None
        if filename.endswith('jpeg'):
            img = load_img(path=direct+filename, target_size=(224,224))
        elif filename.endswith('jpg'):
            img = load_img(path=direct+filename, target_size=(224,224))
        if img is not None :
            img = img_to_array(img)
            images.append(img)
            if(filename.startswith('dog')):
                y.append(1)
            else:
                y.append(0)
    return images, y

In [5]:
images, labels = load_images_from_folder('dataset/')
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(535, 224, 224, 3)
(535,)


In [12]:
import sklearn.cross_validation as cv
X_train, X_test, Y_train, Y_test = cv.train_test_split(images, labels, test_size=0.24)

In [13]:
base_model = VGG19(weights = "imagenet", include_top=False, input_shape=(224,224,3))

In [14]:
for layer in base_model.layers[:23]:
    layer.trainable = False

In [15]:
x = base_model.output
x = Flatten()(x)
x = Dense(200, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(1, activation='sigmoid')(x)

# creating the final model 
model_final = Model(input = base_model.input, output = predictions)

/Users/prachigarg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [16]:
model_final.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [17]:
model_final.fit(X_train, Y_train,epochs=50, batch_size=16,
            validation_data=(X_test, Y_test))

Train on 406 samples, validate on 129 samples
Epoch 1/50
406/406 [==============================] - 261s 643ms/step - loss: 1.6481 - acc: 0.8424 - val_loss: 0.4135 - val_acc: 0.9690
Epoch 2/50
406/406 [==============================] - 263s 648ms/step - loss: 0.6506 - acc: 0.9458 - val_loss: 0.4945 - val_acc: 0.9612
Epoch 3/50
406/406 [==============================] - 278s 684ms/step - loss: 0.2611 - acc: 0.9729 - val_loss: 0.5155 - val_acc: 0.9535
Epoch 4/50
406/406 [==============================] - 300s 738ms/step - loss: 0.2010 - acc: 0.9803 - val_loss: 0.5309 - val_acc: 0.9612
Epoch 5/50
406/406 [==============================] - 290s 715ms/step - loss: 0.3738 - acc: 0.9729 - val_loss: 0.5693 - val_acc: 0.9612
Epoch 6/50
406/406 [==============================] - 296s 730ms/step - loss: 0.2012 - acc: 0.9852 - val_loss: 0.3917 - val_acc: 0.9612
Epoch 7/50
406/406 [==============================] - 314s 774ms/step - loss: 0.1628 - acc: 0.9877 - val_loss: 0.5773 - val_acc: 0.9535
Ep

KeyboardInterrupt: 